# Website Scrapping - Udemy

* 수집 데이터 수 : 키워드당 100개
* 타겟 : 페이지당 20개의 데이터를 조회할 수 있고, 총 5페이지를 순회합니다.

평점을 받지 못한 강의가 있는 페이지는 건너뜁니다. 이 때 20개의 데이터가 일괄 누락되지만,
리뷰 순으로 검색하므로 누락 값에 대한 정확도 감소 폭은 적습니다.

-----

### udemy_scrapper() 동작 순서
1. selenium webdrive를 통해 Udemy 사이트에 접속합니다.
2. 언어 설정을 한국어 -> **영어**로 변경합니다.
3. Input 값으로 입력한 Keyword를 검색합니다.
4. Filter를 **Most-Reviewed**로 변경합니다.
5. 총 5페이지를 순회하며 데이터를 순차적으로 저장합니다.
6. score 데이터는 '(00.000)' 형태의 문자열로 저장되었습니다. list comprehension을 통해 int형으로 변환합니다
7. score 순으로 정렬되지 않은 경우가 있습니다. 내림차순으로 다시 한번 정렬해줍니다.


In [115]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd

### Test Code

In [327]:
# 크롬 드라이버 할당, 페이지 열기
drive = webdriver.Chrome(executable_path='../chromedriver')
link = "https://www.udemy.com/courses/search/?courseLabel=6594&p=1&q=python&sort=most-reviewed"
drive.get(link)

In [358]:
# 필요한 데이터 스크랩핑 (CSS Selector 활용)
li = []
li = drive.find_elements_by_css_selector('.udlite-focus-visible-target')
link_li = drive.find_elements_by_css_selector('.udlite-custom-focus-visible.browse-course-card--link--3KIkQ')

In [365]:
link_li[0].get_attribute('href')

'https://www.udemy.com/course/machinelearning/'

### Udemy 강의 - 강의명, 강의설명, 강의소개 사이트 링크, 평점 수집

In [366]:
def udemy_scrapper(keyword):
    drive = webdriver.Chrome(executable_path='../chromedriver')
    #데이터프레임
    title = []
    summary = []
    score = []
    course_url = []
    link = 'https://www.udemy.com'
    drive.get(link)
    drive.implicitly_wait(10)
    language = drive.find_element_by_css_selector('option[value="en_US"]')
    language.click()
    for page in range(1,6):
        link = 'https://www.udemy.com/courses/search/?p={}&q={}&sort=most-reviewed'.format(page,keyword)
        print('데이터를 수집중입니다. (진행률 : {}/5)'.format(page))
        drive.get(link)
        drive.implicitly_wait(10)
        title_li = drive.find_elements_by_css_selector('.udlite-focus-visible-target')
        subtitle_li = drive.find_elements_by_css_selector('.udlite-text-sm.course-card--course-headline--yIrRk')
        score_li = drive.find_elements_by_css_selector('.udlite-text-xs.course-card--reviews-text--12UpL')
        link_li = drive.find_elements_by_css_selector('.udlite-custom-focus-visible.browse-course-card--link--3KIkQ')
        if len(title_li) == len(score_li):
            for i in range(len(title_li)):
                # 데이터시리즈 생성
                score.append(score_li[i].text)
                title.append(title_li[i].text)
                summary.append(subtitle_li[i].text)
                course_url.append(link_li[i].get_attribute('href'))
        else:
            print('평점이 부족합니다.')
            continue
        # score -> int형으로 변환
    score = [int(i[1:-1].replace(',','')) for i in score] 
    global udemy_df
    udemy_df= pd.DataFrame(list(zip(title,summary,course_url,score)), columns = ['Title','Summary','Link','Score'])
    # udemy_df 정렬
    udemy_df = udemy_df.sort_values(by='Score', ascending=False)
    drive.quit()

### Keyword : Python 

In [367]:
udemy_scrapper('Python')

데이터를 수집중입니다. (진행률 : 1/5)
데이터를 수집중입니다. (진행률 : 2/5)
데이터를 수집중입니다. (진행률 : 3/5)
데이터를 수집중입니다. (진행률 : 4/5)
데이터를 수집중입니다. (진행률 : 5/5)


In [368]:
udemy_df

,Title,Summary,Link,Score
0,2020 Complete Python Bootcamp: From Zero to He...,Learn Python like a Professional! Start from t...,https://www.udemy.com/course/complete-python-b...,316595
1,Machine Learning A-Z™: Hands-On Python & R In ...,Learn to create Machine Learning Algorithms in...,https://www.udemy.com/course/machinelearning/,131812
2,Python for Data Science and Machine Learning B...,"Learn how to use NumPy, Pandas, Seaborn , Matp...",https://www.udemy.com/course/python-for-data-s...,86960
3,The Data Science Course 2020: Complete Data Sc...,"Complete Data Science Training: Mathematics, S...",https://www.udemy.com/course/the-data-science-...,76353
4,Automate the Boring Stuff with Python Programming,A practical programming course for office work...,https://www.udemy.com/course/automate/,70072
...,...,...,...,...
95,Complete Python 3 Masterclass Journey,Master Python 3! Use story based learning to g...,https://www.udemy.com/course/complete-python-3...,2427
96,Learn Python MADE EASY : A Concise Python Cour...,Python for beginners. Python for data science....,https://www.udemy.com/course/python-programmin...,2398
97,Modern Deep Learning in Python,"Build with modern libraries like Tensorflow, T...",https://www.udemy.com/course/data-science-deep...,2364
99,Complete 2020 Data Science & Machine Learning ...,"Learn Python, Tensorflow, Deep Learning, Regre...",https://www.udemy.com/course/python-data-scien...,2245


In [369]:
udemy_df.to_excel('./udemy_python.xlsx')

### Keyword : Security

In [370]:
udemy_scrapper('security')

데이터를 수집중입니다. (진행률 : 1/5)
데이터를 수집중입니다. (진행률 : 2/5)
데이터를 수집중입니다. (진행률 : 3/5)
데이터를 수집중입니다. (진행률 : 4/5)
데이터를 수집중입니다. (진행률 : 5/5)


In [371]:
udemy_df.head()

,Title,Summary,Link,Score
0,Learn Ethical Hacking From Scratch,Become an ethical hacker that can hack compute...,https://www.udemy.com/course/learn-ethical-hac...,83896
9,Spring & Hibernate for Beginners (includes Spr...,"Spring 5: Learn Spring 5 Core, AOP, Spring MVC...",https://www.udemy.com/course/spring-hibernate-...,45115
14,The Complete Cyber Security Course : Hackers E...,"Volume 1 : Become a Cyber Security Specialist,...",https://www.udemy.com/course/the-complete-inte...,30386
32,Practical Ethical Hacking - The Complete Course,2020 Launch! Learn how to hack like a pro by a...,https://www.udemy.com/course/practical-ethical...,17389
12,Information Security Awareness-ISO 27001:2013-...,Information Security Literate with latest upda...,https://www.udemy.com/course/isms-awareness/,13536


In [372]:
udemy_df.to_excel('./udemy_security.xlsx')

### Keyword : Deep Learning

In [373]:
udemy_scrapper('Deep learning')

데이터를 수집중입니다. (진행률 : 1/5)
데이터를 수집중입니다. (진행률 : 2/5)
평점이 부족합니다.
데이터를 수집중입니다. (진행률 : 3/5)
데이터를 수집중입니다. (진행률 : 4/5)
데이터를 수집중입니다. (진행률 : 5/5)


In [374]:
udemy_df.head()

,Title,Summary,Link,Score
15,The Data Science Course 2020: Complete Data Sc...,"Complete Data Science Training: Mathematics, S...",https://www.udemy.com/course/the-data-science-...,76353
4,Deep Learning A-Z™: Hands-On Artificial Neural...,Learn to create Deep Learning Algorithms in Py...,https://www.udemy.com/course/deeplearning/,33342
3,Deep Learning Prerequisites: The Numpy Stack i...,"The Numpy, Scipy, Pandas, and Matplotlib stack...",https://www.udemy.com/course/deep-learning-pre...,18357
16,Artificial Intelligence A-Z™: Learn How To Bui...,"Combine the power of Data Science, Machine Lea...",https://www.udemy.com/course/artificial-intell...,16292
14,NLP - Natural Language Processing with Python,"Learn to use Machine Learning, Spacy, NLTK, Sc...",https://www.udemy.com/course/nlp-natural-langu...,5795


In [375]:
udemy_df.to_excel('./udemy_DeepLearning.xlsx')

### Keyword : JavaScript

In [376]:
udemy_scrapper('JavaScript')

데이터를 수집중입니다. (진행률 : 1/5)
데이터를 수집중입니다. (진행률 : 2/5)
데이터를 수집중입니다. (진행률 : 3/5)
데이터를 수집중입니다. (진행률 : 4/5)
데이터를 수집중입니다. (진행률 : 5/5)


In [377]:
udemy_df.head(10)

,Title,Summary,Link,Score
29,The Web Developer Bootcamp,The only course you need to learn web developm...,https://www.udemy.com/course/the-web-developer...,182487
25,"React - The Complete Guide (incl Hooks, React ...",Dive in and learn React.js from scratch! Learn...,https://www.udemy.com/course/react-the-complet...,90276
28,The Complete JavaScript Course 2020: Build Rea...,Master JavaScript with the most complete cours...,https://www.udemy.com/course/the-complete-java...,83059
41,The Complete Web Developer Course 2.0,Learn Web Development by building 25 websites ...,https://www.udemy.com/course/the-complete-web-...,63041
21,The Complete Web Developer in 2020: Zero to Ma...,Learn to code and become a Web Developer in 20...,https://www.udemy.com/course/the-complete-web-...,40776
11,"Vue - The Complete Guide (w/ Router, Vuex, Com...",Vue.js is an awesome JavaScript Framework for ...,https://www.udemy.com/course/vuejs-2-the-compl...,40721
6,JavaScript: Understanding the Weird Parts,An advanced JavaScript course for everyone! Sc...,https://www.udemy.com/course/understand-javasc...,39033
42,Python and Django Full Stack Web Developer Boo...,"Learn to build websites with HTML , CSS , Boot...",https://www.udemy.com/course/python-and-django...,33496
32,Learn and Understand NodeJS,"Dive deep under the hood of NodeJS. Learn V8, ...",https://www.udemy.com/course/understand-nodejs/,23853
7,Modern JavaScript From The Beginning,Learn and build projects with pure JavaScript ...,https://www.udemy.com/course/modern-javascript...,21185


In [378]:
udemy_df.to_excel('./udemy_JavaScript.xlsx')

### Keyword : Spark

In [381]:
udemy_scrapper('Spark')

데이터를 수집중입니다. (진행률 : 1/5)
데이터를 수집중입니다. (진행률 : 2/5)
데이터를 수집중입니다. (진행률 : 3/5)
데이터를 수집중입니다. (진행률 : 4/5)
데이터를 수집중입니다. (진행률 : 5/5)


In [382]:
udemy_df.head(20)

,Title,Summary,Link,Score
5,The Ultimate Hands-On Hadoop - Tame your Big D...,"Hadoop tutorial with MapReduce, HDFS, Spark, F...",https://www.udemy.com/course/the-ultimate-hand...,21527
2,Spark and Python for Big Data with PySpark,"Learn how to use Spark with Python, including ...",https://www.udemy.com/course/spark-and-python-...,12340
3,Apache Spark with Scala - Hands On with Big Data!,Dive right in with 20+ hands-on examples of an...,https://www.udemy.com/course/apache-spark-with...,11758
4,Taming Big Data with Apache Spark and Python -...,Dive right in with 20+ hands-on examples of an...,https://www.udemy.com/course/taming-big-data-w...,9096
1,Scala and Spark for Big Data and Machine Learning,Learn the latest Big Data technology - Spark a...,https://www.udemy.com/course/scala-and-spark-f...,4348
91,Spark Starter Kit,"NOT another ""What is Spark?"" course ! Explore ...",https://www.udemy.com/course/sparkstarterkit/,3178
7,Learn Big Data: The Hadoop Ecosystem Masterclass,"Master the Hadoop ecosystem using HDFS, MapRed...",https://www.udemy.com/course/learn-big-data-th...,3161
0,Streaming Big Data with Spark Streaming & Scal...,Hands-on examples of processing massive stream...,https://www.udemy.com/course/taming-big-data-w...,2749
6,Apache Spark 2.0 with Java -Learn Spark from a...,Learn analyzing large data sets with Apache Sp...,https://www.udemy.com/course/apache-spark-cour...,2501
8,CCA 175 - Spark and Hadoop Developer Certifica...,Cloudera Certified Associate Spark and Hadoop ...,https://www.udemy.com/course/cca-175-spark-and...,2015


In [383]:
udemy_df.to_excel('./udemy_Spark.xlsx')

### Keyword : Power BI

In [384]:
udemy_scrapper('power bi')

데이터를 수집중입니다. (진행률 : 1/5)
데이터를 수집중입니다. (진행률 : 2/5)
데이터를 수집중입니다. (진행률 : 3/5)
데이터를 수집중입니다. (진행률 : 4/5)
데이터를 수집중입니다. (진행률 : 5/5)


In [385]:
udemy_df.head(20)

,Title,Summary,Link,Score
0,Microsoft Power BI - A Complete Introduction [...,"Learn how to use Microsoft's Power BI Tools, i...",https://www.udemy.com/course/powerbi-complete-...,38464
1,Microsoft Power BI - Up & Running With Power B...,"Transform, analyze & visualize data with Micro...",https://www.udemy.com/course/microsoft-power-b...,24043
2,Power BI A-Z: Hands-On Power BI Training For D...,Learn Microsoft Power BI for Data Science and ...,https://www.udemy.com/course/mspowerbi/,9822
3,Complete Introduction to Microsoft Power BI [2...,Learn to use the free Microsoft Power BI to cr...,https://www.udemy.com/course/powerful-reports-...,6593
4,Microsoft Power BI - Business Intelligence for...,Beginner Course on Power BI : Business Intelli...,https://www.udemy.com/course/business-data-ana...,3263
5,Power BI Masterclass - beginners to advanced,Microsoft Power BI - How to master self servic...,https://www.udemy.com/course/power-bi-mastercl...,3232
6,Power BI Master Class-Data Models and DAX Form...,"Learn how to create data models, relationships...",https://www.udemy.com/course/power-bi-master-c...,2975
7,Power BI - Data Analytics Essentials with Powe...,"Get Started Quickly with Visualization, Data P...",https://www.udemy.com/course/data-analytics-es...,2012
8,Power BI Master Class - Dashboards and Power B...,Learn how to create interactive dashboards and...,https://www.udemy.com/course/power-bi-master-c...,1992
9,Business Data Analysis with Microsoft Excel an...,An in-depth training on Excel and Power BI for...,https://www.udemy.com/course/business-data-ana...,1938


In [386]:
udemy_df.to_excel('./udemy_PowerBI.xlsx')

### Keyword : Machine Learning

In [387]:
udemy_scrapper('machine learning')

데이터를 수집중입니다. (진행률 : 1/5)
평점이 부족합니다.
데이터를 수집중입니다. (진행률 : 2/5)
데이터를 수집중입니다. (진행률 : 3/5)
데이터를 수집중입니다. (진행률 : 4/5)
데이터를 수집중입니다. (진행률 : 5/5)


In [388]:
udemy_df.head(20)

,Title,Summary,Link,Score
13,Machine Learning A-Z™: Hands-On Python & R In ...,Learn to create Machine Learning Algorithms in...,https://www.udemy.com/course/machinelearning/,131812
14,Python for Data Science and Machine Learning B...,"Learn how to use NumPy, Pandas, Seaborn , Matp...",https://www.udemy.com/course/python-for-data-s...,86960
15,The Data Science Course 2020: Complete Data Sc...,"Complete Data Science Training: Mathematics, S...",https://www.udemy.com/course/the-data-science-...,76353
16,The Complete Web Developer in 2020: Zero to Ma...,Learn to code and become a Web Developer in 20...,https://www.udemy.com/course/the-complete-web-...,40776
17,Deep Learning A-Z™: Hands-On Artificial Neural...,Learn to create Deep Learning Algorithms in Py...,https://www.udemy.com/course/deeplearning/,33342
18,"Machine Learning, Data Science and Deep Learni...",Complete hands-on machine learning tutorial wi...,https://www.udemy.com/course/data-science-and-...,22508
19,Complete Python Developer in 2020: Zero to Mas...,How to become a Python 3 Developer and get hir...,https://www.udemy.com/course/complete-python-d...,19819
20,Deep Learning Prerequisites: The Numpy Stack i...,"The Numpy, Scipy, Pandas, and Matplotlib stack...",https://www.udemy.com/course/deep-learning-pre...,18357
21,"Python Bootcamp: Vom Anfänger zum Profi, inkl....","✓ Python komplett: Grundlagen, Webapps, Crawle...",https://www.udemy.com/course/python-bootcamp/,16848
22,Artificial Intelligence A-Z™: Learn How To Bui...,"Combine the power of Data Science, Machine Lea...",https://www.udemy.com/course/artificial-intell...,16292


In [389]:
udemy_df.to_excel('./udemy_MachineLearning.xlsx')

### Keyword : AWS

In [390]:
udemy_scrapper('aws')

데이터를 수집중입니다. (진행률 : 1/5)
데이터를 수집중입니다. (진행률 : 2/5)
데이터를 수집중입니다. (진행률 : 3/5)
데이터를 수집중입니다. (진행률 : 4/5)
데이터를 수집중입니다. (진행률 : 5/5)


In [391]:
udemy_df.head(20)

,Title,Summary,Link,Score
33,AWS Certified Solutions Architect - Associate ...,Want to pass the AWS Solutions Architect - Ass...,https://www.udemy.com/course/aws-certified-sol...,189790
7,Ultimate AWS Certified Solutions Architect Ass...,Pass the AWS Certified Solutions Architect Ass...,https://www.udemy.com/course/aws-certified-sol...,43675
41,AWS Certified Developer - Associate 2020,Do you want AWS certification? Do you want to ...,https://www.udemy.com/course/aws-certified-dev...,37617
5,Ultimate AWS Certified Developer Associate 202...,Become an AWS Certified Developer! Learn all A...,https://www.udemy.com/course/aws-certified-dev...,26305
38,Amazon Web Services (AWS) Certified - 4 Certif...,"Videos, labs & practice exams - AWS Certified ...",https://www.udemy.com/course/aws-certified-ass...,16332
13,AWS Certified Solutions Architect Associate Pr...,Be an AWS Solutions Architect! AWS Certified S...,https://www.udemy.com/course/aws-certified-sol...,15630
34,AWS Certified Cloud Practitioner 2020,Learn AWS from scratch from one of the top rat...,https://www.udemy.com/course/aws-certified-clo...,13697
2,[NEW] Ultimate AWS Certified Cloud Practitione...,Pass the AWS Certified Cloud Practitioner Cert...,https://www.udemy.com/course/aws-certified-clo...,13089
37,AWS Certified SysOps Administrator - Associate...,Want to pass the AWS Certified SysOps Administ...,https://www.udemy.com/course/aws-certified-sys...,11488
32,AWS Certified Cloud Practitioner: 6 Full Pract...,Pass your Amazon Web Services Certified Cloud ...,https://www.udemy.com/course/aws-certified-clo...,10370


In [392]:
udemy_df.to_excel('./udemy_aws.xlsx')

### Keyword : Time Series

In [414]:
udemy_scrapper('time series')

데이터를 수집중입니다. (진행률 : 1/5)
데이터를 수집중입니다. (진행률 : 2/5)
데이터를 수집중입니다. (진행률 : 3/5)
평점이 부족합니다.
데이터를 수집중입니다. (진행률 : 4/5)
데이터를 수집중입니다. (진행률 : 5/5)
평점이 부족합니다.


In [415]:
udemy_df.head(20)

,Title,Summary,Link,Score
0,Python for Time Series Data Analysis,"Learn how to use Python , Pandas, Numpy , and ...",https://www.udemy.com/course/python-for-time-s...,3521
1,Tensorflow 2.0: Deep Learning and Artificial I...,"Neural Networks for Computer Vision, Time Seri...",https://www.udemy.com/course/deep-learning-ten...,3482
2,Deep Learning: Recurrent Neural Networks in Py...,"GRU, LSTM, Time Series Forecasting, Stock Pred...",https://www.udemy.com/course/deep-learning-rec...,2823
3,Introduction to Time Series Analysis and Forec...,Work with time series and all sorts of time re...,https://www.udemy.com/course/time-series-analy...,1834
4,Time Series Analysis in Python 2020,"Time Series Analysis in Python: Theory, Modeli...",https://www.udemy.com/course/time-series-analy...,820
33,Complete Time Series Analysis With Python,"Python Time Series: How To Use Data Science, S...",https://www.udemy.com/course/complete-time-ser...,620
5,Machine Learning & Deep Learning in Python & R,"Covers Regression, Decision Trees, SVM, Neural...",https://www.udemy.com/course/data_science_a_to_z/,587
6,Marketing Analytics: Forecasting Models with E...,Master Marketing Analytics| Forecasting and Ti...,https://www.udemy.com/course/marketing-analyti...,576
7,Forecasting and Time Series Analysis in Tableau,"Use Tableau to work with time series, generate...",https://www.udemy.com/course/forecasting-and-t...,491
8,Time Series Analysis and Forecasting using Python,Learn about time series analysis & forecasting...,https://www.udemy.com/course/machine-learning-...,458


In [416]:
udemy_df.to_excel('./udemy_TimeSeries.xlsx')

### Keyword : Algorithm

In [396]:
udemy_scrapper('algorithm')

데이터를 수집중입니다. (진행률 : 1/5)
데이터를 수집중입니다. (진행률 : 2/5)
데이터를 수집중입니다. (진행률 : 3/5)
데이터를 수집중입니다. (진행률 : 4/5)
데이터를 수집중입니다. (진행률 : 5/5)


In [397]:
udemy_df.head(20)

,Title,Summary,Link,Score
0,Machine Learning A-Z™: Hands-On Python & R In ...,Learn to create Machine Learning Algorithms in...,https://www.udemy.com/course/machinelearning/,131812
1,Deep Learning A-Z™: Hands-On Artificial Neural...,Learn to create Deep Learning Algorithms in Py...,https://www.udemy.com/course/deeplearning/,33342
2,Python for Financial Analysis and Algorithmic ...,"Learn numpy , pandas , matplotlib , quantopian...",https://www.udemy.com/course/python-for-financ...,13457
3,Mastering Data Structures & Algorithms using C...,"Learn, Analyse and Implement Data Structure us...",https://www.udemy.com/course/datastructurescncpp/,11955
4,The Coding Interview Bootcamp: Algorithms + Da...,Ace your next Javascript coding interview by m...,https://www.udemy.com/course/coding-interview-...,10662
5,JavaScript Algorithms and Data Structures Mast...,The Missing Computer Science and Coding Interv...,https://www.udemy.com/course/js-algorithms-and...,10307
6,Master the Coding Interview: Data Structures +...,Ultimate coding interview bootcamp. Get more j...,https://www.udemy.com/course/master-the-coding...,9668
7,"Python for Data Structures, Algorithms, and In...",Get a kick start on your career and ace your c...,https://www.udemy.com/course/python-for-data-s...,8277
8,Data Structures and Algorithms: Deep Dive Usin...,"Learn about Arrays, Linked Lists, Trees, Hasht...",https://www.udemy.com/course/data-structures-a...,6996
9,Practical Data Structures & Algorithms in Java...,Go from zero to hero in the most important alg...,https://www.udemy.com/course/practical-data-st...,4515


In [398]:
udemy_df.to_excel('./udemy_Algorithm.xlsx')

### Keyword : Computer Vision

In [399]:
udemy_scrapper('computer vision')

데이터를 수집중입니다. (진행률 : 1/5)
데이터를 수집중입니다. (진행률 : 2/5)
데이터를 수집중입니다. (진행률 : 3/5)
데이터를 수집중입니다. (진행률 : 4/5)
평점이 부족합니다.
데이터를 수집중입니다. (진행률 : 5/5)


In [400]:
udemy_df.head(20)

,Title,Summary,Link,Score
0,Deep Learning and Computer Vision A-Z™: OpenCV...,Become a Wizard of all the latest Computer Vis...,https://www.udemy.com/course/computer-vision-a-z/,5061
1,Python for Computer Vision with OpenCV and Dee...,Learn the latest techniques in computer vision...,https://www.udemy.com/course/python-for-comput...,4686
2,Tensorflow 2.0: Deep Learning and Artificial I...,"Neural Networks for Computer Vision, Time Seri...",https://www.udemy.com/course/deep-learning-ten...,3482
3,Master Computer Vision™ OpenCV4 in Python with...,"Master OpenCV4 like a pro while learning Dlib,...",https://www.udemy.com/course/master-computer-v...,3220
4,"Deep Learning: Advanced Computer Vision (GANs,...","VGG, ResNet, Inception, SSD, RetinaNet, Neural...",https://www.udemy.com/course/advanced-computer...,3150
5,The Complete Self-Driving Car Course - Applied...,"Learn to use Deep Learning, Computer Vision an...",https://www.udemy.com/course/applied-deep-lear...,2725
6,"Deep Learning Computer Vision™ CNN, OpenCV, YO...",2020 Update with TensorFlow 2.0 Support. Becom...,https://www.udemy.com/course/master-deep-learn...,1436
7,PyTorch for Deep Learning and Computer Vision,Build Highly Sophisticated Deep Learning and C...,https://www.udemy.com/course/pytorch-for-deep-...,1181
8,Autonomous Cars: Deep Learning and Computer Vi...,"Learn OpenCV, Keras, object and lane detection...",https://www.udemy.com/course/autonomous-cars-d...,666
36,[2020] Python tutorial from Zero to Hero: + Ma...,"Includes Machine Learning, Artificial Intellig...",https://www.udemy.com/course/2020-python-tutor...,525


In [401]:
udemy_df.to_excel('./udemy_ComputerVision.xlsx')

### Keyword : Tensorflow

In [402]:
udemy_scrapper('tensorflow')

데이터를 수집중입니다. (진행률 : 1/5)
데이터를 수집중입니다. (진행률 : 2/5)
데이터를 수집중입니다. (진행률 : 3/5)
데이터를 수집중입니다. (진행률 : 4/5)
데이터를 수집중입니다. (진행률 : 5/5)


In [403]:
udemy_df.head(20)

,Title,Summary,Link,Score
3,Python for Data Science and Machine Learning B...,"Learn how to use NumPy, Pandas, Seaborn , Matp...",https://www.udemy.com/course/python-for-data-s...,86960
4,"Machine Learning, Data Science and Deep Learni...",Complete hands-on machine learning tutorial wi...,https://www.udemy.com/course/data-science-and-...,22508
5,Complete Guide to TensorFlow for Deep Learning...,Learn how to use Google's Deep Learning Framew...,https://www.udemy.com/course/complete-guide-to...,15085
6,Data Science: Deep Learning in Python,The MOST in-depth look at neural network theor...,https://www.udemy.com/course/data-science-deep...,6838
7,GCP: Complete Google Data Engineer and Cloud A...,"The Google Cloud for ML with TensorFlow, Big D...",https://www.udemy.com/course/gcp-data-engineer...,5326
8,Complete Machine Learning and Data Science: Ze...,"Learn Data Science, Data Analysis, Machine Lea...",https://www.udemy.com/course/complete-machine-...,4916
9,Deep Learning and NLP A-Z™: How to create a Ch...,Learn the Theory and How to implement state of...,https://www.udemy.com/course/chatbot/,3498
10,Tensorflow 2.0: Deep Learning and Artificial I...,"Neural Networks for Computer Vision, Time Seri...",https://www.udemy.com/course/deep-learning-ten...,3482
11,Master Computer Vision™ OpenCV4 in Python with...,"Master OpenCV4 like a pro while learning Dlib,...",https://www.udemy.com/course/master-computer-v...,3220
12,"Deep Learning: Advanced Computer Vision (GANs,...","VGG, ResNet, Inception, SSD, RetinaNet, Neural...",https://www.udemy.com/course/advanced-computer...,3150


In [404]:
udemy_df.to_excel('./udemy_Tensorflow.xlsx')

### Keyword : Game

In [405]:
udemy_scrapper('game')

데이터를 수집중입니다. (진행률 : 1/5)
데이터를 수집중입니다. (진행률 : 2/5)
평점이 부족합니다.
데이터를 수집중입니다. (진행률 : 3/5)
데이터를 수집중입니다. (진행률 : 4/5)
데이터를 수집중입니다. (진행률 : 5/5)


In [406]:
udemy_df.head(20)

,Title,Summary,Link,Score
20,Discovering Godot: Make Video Games in Python-...,Game development with Godot Engine 3.0 & 3.1 :...,https://www.udemy.com/course/godot/,3196
21,Game Music Composition: Make Music For Games F...,Learn to compose music for any and every type ...,https://www.udemy.com/course/gamemusiccourse/,2988
22,The Ultimate Guide to 2D Mobile Game Developme...,Created in partnership with Unity Technologies...,https://www.udemy.com/course/the-ultimate-guid...,2954
23,Learn To Create An RPG Game In Unity,Game development made easy. Learn C# using Uni...,https://www.udemy.com/course/unity2drpg/,2906
24,Unreal Engine Blueprint Game Developer,Code Your First Video Game in 100 minutes with...,https://www.udemy.com/course/unrealblueprint/,2597
25,Learn To Code By Making a 2D Platformer in Unity,Game development made easy. Learn C# using Uni...,https://www.udemy.com/course/unity2dplatformer/,2448
26,Shader Development from Scratch for Unity with Cg,Learn to program the graphics pipeline in Unit...,https://www.udemy.com/course/unity-shaders/,2269
27,Blender Character Creator v2.0 for Video Games...,Model Video Game Characters. Use The Sculpt To...,https://www.udemy.com/course/blendercharacters/,2096
28,Build Your Own First Person Shooter / Survival...,Part I: Create and Design Artificially Intelli...,https://www.udemy.com/course/build-your-own-fi...,1975
29,Learn Professional 2D Game Graphic Design in P...,Learn how you can create your own professional...,https://www.udemy.com/course/create-quick-prof...,1916


In [407]:
udemy_df.to_excel('./udemy_game.xlsx')

### Keyword : Hadoop

In [408]:
udemy_scrapper('hadoop')

데이터를 수집중입니다. (진행률 : 1/5)
데이터를 수집중입니다. (진행률 : 2/5)
데이터를 수집중입니다. (진행률 : 3/5)
데이터를 수집중입니다. (진행률 : 4/5)
데이터를 수집중입니다. (진행률 : 5/5)
평점이 부족합니다.


In [409]:
udemy_df.head(20)

,Title,Summary,Link,Score
0,The Ultimate Hands-On Hadoop - Tame your Big D...,"Hadoop tutorial with MapReduce, HDFS, Spark, F...",https://www.udemy.com/course/the-ultimate-hand...,21527
11,Hadoop Starter Kit,"Hadoop learning made easy and fun. Learn HDFS,...",https://www.udemy.com/course/hadoopstarterkit/,14761
66,Big Data and Hadoop Essentials,Essential Knowledge for everyone associated wi...,https://www.udemy.com/course/big-data-and-hado...,11668
1,Learn Big Data: The Hadoop Ecosystem Masterclass,"Master the Hadoop ecosystem using HDFS, MapRed...",https://www.udemy.com/course/learn-big-data-th...,3161
15,Taming Big Data with MapReduce and Hadoop - Ha...,Learn MapReduce fast by building over 10 real ...,https://www.udemy.com/course/taming-big-data-w...,2442
17,Hive to ADVANCE Hive (Real time usage) :Hadoop...,In and Out of Apache Hive - From Basic Hive to...,https://www.udemy.com/course/hadoop-querying-t...,2359
2,CCA 175 - Spark and Hadoop Developer Certifica...,Cloudera Certified Associate Spark and Hadoop ...,https://www.udemy.com/course/cca-175-spark-and...,2015
7,Big Data and Hadoop for Beginners - with Hands...,"Everything you need to know about Big Data, an...",https://www.udemy.com/course/big-data-and-hado...,1925
22,Learning Apache Hadoop EcoSystem- Hive,Learn Apache Hive and Start working with SQL q...,https://www.udemy.com/course/learning-apache-h...,1684
72,Big data Internship Program - Foundation,A Complete Guide to Learn Big data and Hadoop ...,https://www.udemy.com/course/big-data-internsh...,1350


In [410]:
udemy_df.to_excel('./udemy_Hadoop.xlsx')

### Keyword : Unity

In [417]:
udemy_scrapper('unity')

데이터를 수집중입니다. (진행률 : 1/5)
데이터를 수집중입니다. (진행률 : 2/5)
데이터를 수집중입니다. (진행률 : 3/5)
데이터를 수집중입니다. (진행률 : 4/5)
데이터를 수집중입니다. (진행률 : 5/5)


In [418]:
udemy_df.head(20)

,Title,Summary,Link,Score
0,Complete C# Unity Game Developer 2D,Learn Unity in C# & Code Your First Seven 2D V...,https://www.udemy.com/course/unitycourse/,78769
1,Complete C# Unity Game Developer 3D,Design & Develop Video Games. Learn C# in Unit...,https://www.udemy.com/course/unitycourse2/,22219
2,The Ultimate Guide to Game Development with Un...,Created in partnership with Unity Technologies...,https://www.udemy.com/course/the-ultimate-guid...,12996
3,Complete C# Masterclass,"Learn C# Programming - WPF, Databases, Linq, C...",https://www.udemy.com/course/complete-csharp-m...,8994
4,RPG Core Combat Creator: Learn Intermediate Un...,Build Combat for Role Playing Game (RPG) in Un...,https://www.udemy.com/course/unityrpg/,7455
5,Become a Game Designer the Complete Series Cod...,"Master Unity, 3D game design, 2D game design, ...",https://www.udemy.com/course/become-a-game-des...,6264
6,Unity Game Development Build 2D & 3D Games,Learn Unity game design & 2D & 3D game develop...,https://www.udemy.com/course/devslopes-unity3d/,4946
7,Master Unity By Building 6 Fully Featured Game...,"Learn How To Plan, Design, Create And Publish ...",https://www.udemy.com/course/make-mobile-games...,4079
8,The Ultimate Guide to 2D Mobile Game Developme...,Created in partnership with Unity Technologies...,https://www.udemy.com/course/the-ultimate-guid...,2954
9,Learn To Create An RPG Game In Unity,Game development made easy. Learn C# using Uni...,https://www.udemy.com/course/unity2drpg/,2906


In [419]:
udemy_df.to_excel('./udemy_unity.xlsx')

### Keyword : Unreal

In [420]:
udemy_scrapper('unreal')

데이터를 수집중입니다. (진행률 : 1/5)
데이터를 수집중입니다. (진행률 : 2/5)
데이터를 수집중입니다. (진행률 : 3/5)
데이터를 수집중입니다. (진행률 : 4/5)
평점이 부족합니다.
데이터를 수집중입니다. (진행률 : 5/5)


In [421]:
udemy_df.head(20)

,Title,Summary,Link,Score
0,Unreal Engine C++ Developer: Learn C++ and Mak...,Created in collaboration with Epic Games. Lear...,https://www.udemy.com/course/unrealcourse/,45679
65,Unreal Engine 4: How to Develop Your First Two...,"Epic Games-Approved: learn the Unreal Editor, ...",https://www.udemy.com/course/unreale4/,9369
75,Unreal Engine 4: The Complete Beginner's Course,Learn Video Game Development & How To Design a...,https://www.udemy.com/course/unreal-engine-4-t...,5269
73,Unreal Engine 4 Mastery: Create Multiplayer Ga...,Unleash the power of C++ and Blueprint to deve...,https://www.udemy.com/course/unrealengine-cpp/,4982
9,Unreal Engine Blueprint Game Developer,Code Your First Video Game in 100 minutes with...,https://www.udemy.com/course/unrealblueprint/,2597
1,Unreal Engine C++ The Ultimate Game Developer ...,"Learn how to develop, code and package a compl...",https://www.udemy.com/course/unreal-engine-the...,1496
66,Unreal Multiplayer Master: Video Game Dev In C++,Use Unreal for Online Game Dev in C++. Tutoria...,https://www.udemy.com/course/unrealmultiplayer/,1427
74,Unreal Engine 4 Class: Blueprints,Learn Blueprints and create your own games (en...,https://www.udemy.com/course/unreal-engine-4-b...,1418
2,Unreal Engine 4: Project Based VR Course for A...,Use Unreal Engine 4 (free) to turn your 3d pro...,https://www.udemy.com/course/3ds-max-unreal-en...,799
3,Create a Battle Royale game using Unreal Engin...,Create a Battle Royale game like Fortnite with...,https://www.udemy.com/course/battleroyale/,578


In [422]:
udemy_df.to_excel('./udemy_unreal.xlsx')

### Keyword : D3

In [423]:
udemy_scrapper('d3')

데이터를 수집중입니다. (진행률 : 1/5)
데이터를 수집중입니다. (진행률 : 2/5)
평점이 부족합니다.
데이터를 수집중입니다. (진행률 : 3/5)
데이터를 수집중입니다. (진행률 : 4/5)
데이터를 수집중입니다. (진행률 : 5/5)


In [424]:
udemy_df.head(20)

,Title,Summary,Link,Score
0,The Advanced Web Developer Bootcamp,"Learn React 16, Redux, D3, ES2015, Testing, CS...",https://www.udemy.com/course/the-advanced-web-...,8222
1,D3.js Data Visualization Fundamentals - Hands On,Design and build beautiful data visualizations...,https://www.udemy.com/course/d3jsbasics/,3179
2,Mastering data visualization in D3.js [Sept 20...,Design and build beautiful data visualizations...,https://www.udemy.com/course/masteringd3js/,2656
3,Build Data Visualizations with D3.js & Firebase,Learn how to use D3 (v5) & Firebase (Firestore...,https://www.udemy.com/course/build-data-uis-wi...,1084
15,Rapid Prototyping with Ionic: Build a Data-Dri...,Retrieve data from multiple API's; d3js charti...,https://www.udemy.com/course/learn-ionic-frame...,872
18,Data Visualize Data with D3.js The Easy Way,Learn how to use D3.js to create data driven g...,https://www.udemy.com/course/data-visualize-da...,793
4,Learn and Understand D3.js for Data Visualization,Dive deep under the hood of D3.js. Learn core ...,https://www.udemy.com/course/learn-d3js-for-da...,754
20,Master React Native Animations,"Simple, easy to follow, step-by-step lessons f...",https://www.udemy.com/course/master-react-nati...,497
21,"Ruby on Rails 5 - BDD, RSpec and Capybara",Learn behavior-driven development by developin...,https://www.udemy.com/course/ruby-rails-5-bdd-...,454
22,Docker Curso Intensivo para DevOps y Desarroll...,Docker para desarrollar e implementar aplicaci...,https://www.udemy.com/course/curso-intensivo-d...,339


In [425]:
udemy_df.to_excel('./udemy_D3.xlsx')

### Keyword : Kotlin

In [426]:
udemy_scrapper('kotlin')

데이터를 수집중입니다. (진행률 : 1/5)
데이터를 수집중입니다. (진행률 : 2/5)
데이터를 수집중입니다. (진행률 : 3/5)
데이터를 수집중입니다. (진행률 : 4/5)
평점이 부족합니다.
데이터를 수집중입니다. (진행률 : 5/5)
평점이 부족합니다.


In [427]:
udemy_df.head(20)

,Title,Summary,Link,Score
5,The Complete Android Oreo Developer Course - B...,Learn Android O App Development using Java & K...,https://www.udemy.com/course/the-complete-andr...,13341
7,The Complete Android Kotlin Developer Course,"Learn how to build 17 online games, and apps f...",https://www.udemy.com/course/the-complete-kotl...,6725
4,Android App Development Masterclass using Kotlin,Learn Kotlin Android App Development And Becom...,https://www.udemy.com/course/android-oreo-kotl...,3603
3,Kotlin for Android: Beginner to Advanced,"Kotlin is an expressive, concise & powerful de...",https://www.udemy.com/course/devslopes-android...,2366
0,Kotlin for Beginners: Learn Programming With K...,Learn Kotlin from scratch! Grasp object-orient...,https://www.udemy.com/course/kotlin-course/,2308
2,Kotlin for Java Developers,Use your Java skills to learn Kotlin fast. Enh...,https://www.udemy.com/course/kotlin-for-java-d...,1903
1,The Complete Android 10 & Kotlin Development M...,Learn Android 10 App Development From Beginner...,https://www.udemy.com/course/android-kotlin-de...,1479
6,Kotlin for Android O Development: From Beginne...,Learn coding and designing Android apps with K...,https://www.udemy.com/course/kotlinandroid/,1382
8,Kotlin for Android & Java Developers: Clean Co...,Become a professional Kotlin developer and wri...,https://www.udemy.com/course/kotlin-android/,1065
9,Kotlin Android Development Masterclass,"Learn Kotlin, Android Development, Android Stu...",https://www.udemy.com/course/kotlin-android-de...,790


In [428]:
udemy_df.to_excel('./udemy_Kotlin.xlsx')

## 누적 수강생 조회
Score는 평점을 받은 횟수입니다. 강의에 대한 선호도를 확인할 수는 있으나 매출 수준을 그대로 반영하고 있지는 않습니다. 판매지수(누적 수강생 수)를 확인해봅니다

---
### udemy_detail() 동작순서  
1. 만들어진 데이터프레임 중 'Link'컬럼을 순서대로 조회해 해당 사이트로 진입합니다.
2. 해당 페이지의 누적 수강생 수를 수집해 불필요한 텍스트를 제거 후 students_li 리스트에 하나씩 저장합니다.
3. 기존 데이터프레임에 'Enrollment'라는 이름으로 컬럼을 추가합니다.

### Test Code

In [450]:
udemy_df.Link[5]

'https://www.udemy.com/course/the-complete-android-oreo-developer-course/'

In [451]:
drive = webdriver.Chrome(executable_path='../chromedriver')
drive.get('https://www.udemy.com/course/the-complete-android-oreo-developer-course/')

In [452]:
students = drive.find_elements_by_css_selector('div[data-purpose="enrollment"]')

In [453]:
students[1].text

'63,650명의 수강생'

### Enrollment 컬럼 생성

In [456]:
def udemy_detail(df):
    drive = webdriver.Chrome(executable_path='../chromedriver')
    #데이터프레임
    df = df
    students_li = []
    drive.implicitly_wait(10)
    for page in df.Link:
        link = page
        print('데이터를 수집중입니다.')
        drive.get(link)
        drive.implicitly_wait(10)
        students = drive.find_elements_by_css_selector('div[data-purpose="enrollment"]')
        students_li.append(students[-1].text)
        # score -> int형으로 변환
    students_li = [int(i[:-6].replace(',','')) for i in students_li]
    udemy_df['Enrollment'] = students_li
    drive.quit()

In [457]:
udemy_detail(udemy_df)

데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.
데이터를 수집중입니다.


In [459]:
udemy_df.head(20)

,Title,Summary,Link,Score,Enrollment
5,The Complete Android Oreo Developer Course - B...,Learn Android O App Development using Java & K...,https://www.udemy.com/course/the-complete-andr...,13341,63650
7,The Complete Android Kotlin Developer Course,"Learn how to build 17 online games, and apps f...",https://www.udemy.com/course/the-complete-kotl...,6725,122411
4,Android App Development Masterclass using Kotlin,Learn Kotlin Android App Development And Becom...,https://www.udemy.com/course/android-oreo-kotl...,3603,24508
3,Kotlin for Android: Beginner to Advanced,"Kotlin is an expressive, concise & powerful de...",https://www.udemy.com/course/devslopes-android...,2366,11591
0,Kotlin for Beginners: Learn Programming With K...,Learn Kotlin from scratch! Grasp object-orient...,https://www.udemy.com/course/kotlin-course/,2308,10141
2,Kotlin for Java Developers,Use your Java skills to learn Kotlin fast. Enh...,https://www.udemy.com/course/kotlin-for-java-d...,1903,11273
1,The Complete Android 10 & Kotlin Development M...,Learn Android 10 App Development From Beginner...,https://www.udemy.com/course/android-kotlin-de...,1479,7928
6,Kotlin for Android O Development: From Beginne...,Learn coding and designing Android apps with K...,https://www.udemy.com/course/kotlinandroid/,1382,52223
8,Kotlin for Android & Java Developers: Clean Co...,Become a professional Kotlin developer and wri...,https://www.udemy.com/course/kotlin-android/,1065,5133
9,Kotlin Android Development Masterclass,"Learn Kotlin, Android Development, Android Stu...",https://www.udemy.com/course/kotlin-android-de...,790,6479


In [460]:
udemy_df.to_excel('./udemy_Kotlin.xlsx')